# Data analysis 

- Duration of samples and differences
- Plot the mean square energy over time
- Zero-Crossing Rate (ZCR) = Rate the signal changes from pos to neg
- Silence Ratio -> The ratio of pauses in the audio through silence 

In [ ]:
pip install soundfile
pip install librosa

In [28]:
# Data Anlaysis - sample duration 

import numpy as np 
import pandas as pd
import soundfile as sf
import os
import librosa
from pathlib import Path
from itables import init_notebook_mode, show


project_path = os.path.abspath(os.sep) + 'Users/Luis/Desktop/LA/LA/'
train_data_path = project_path + 'ASVspoof2019_LA_train/flac'
train_protocol_path = project_path + 'ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt'

print(train_protocol_path)


train_df = pd.read_csv(train_protocol_path, sep = ' ', header = None, names = ['speaker_id', 'file_name', 'system_id', 'attack_type','key'])
print(f'Found {len(train_df)} files in protocol.')

results = []
train_bon_samples = train_df[train_df['key'] == 'bonafide']['file_name'].iloc[:len(train_df)].tolist()
train_spoofed_samples = train_df[train_df['key'] == 'spoofed']['file_name'].iloc[:len(train_df)].tolist()
train_samples = train_bon_samples + train_spoofed_samples

for filename in train_samples[:10]:
    try:
        file_path = os.path.join(train_data_path, filename + '.flac')
        y, sr = sf.read(file_path, dtype = 'float32')

        if y.ndim > 1:
            y = y.mean(axis=1)
            
        if sr != 16000:
            y = librosa.resample(y, orig_sr = sr, target_sr = 16000)
            sr = 16000

        key = train_df[train_df['file_name'] == filename]['key'].values[0]
        attack = train_df[train_df['file_name'] == filename]['attack_type'].values[0]

        results.append({
            'filename': filename,
            'label': key,
            'attack': attack,
            'duration': len(y)/sr,
            'mean': y.mean(),
            'std': y.std(),
            'max': y.max(),
            'min': y.min()
        })

    except Exception as e:
        print(f"{filename}: {e}")

if results: 
    results_df = pd.DataFrame(results)
    print('\nSummary')
    print(results_df)

C:\Users/Luis/Desktop/LA/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.train.trn.txt
Found 25380 files in protocol.

Summary
       filename     label attack  duration          mean       std       max  \
0  LA_T_1138215  bonafide      -  3.458063  2.995002e-07  0.096481  0.974030   
1  LA_T_1271820  bonafide      -  4.395187 -7.316615e-07  0.096049  0.903534   
2  LA_T_1272637  bonafide      -  2.899500 -5.499374e-07  0.075203  0.817200   
3  LA_T_1276960  bonafide      -  2.812562  4.007886e-07  0.104142  0.999969   
4  LA_T_1341447  bonafide      -  3.533187 -1.041347e-06  0.112211  0.999969   
5  LA_T_1363611  bonafide      -  2.257250 -7.326044e-07  0.109288  0.999969   
6  LA_T_1596451  bonafide      -  2.125563 -5.716045e-07  0.079535  0.575104   
7  LA_T_1608170  bonafide      -  2.003437  4.274651e-07  0.072892  0.899963   
8  LA_T_1684951  bonafide      -  3.791688 -8.833281e-07  0.102231  0.970306   
9  LA_T_1699801  bonafide      -  3.240875  5.614566e-07  0.084541  0.

In [4]:
import soundfile as sf
data, sr = sf.read(file_path)
print("Loaded with soundfile!")

Loaded with soundfile!


In [7]:
list_of_samples = []
for file in os.listdir(train_data_path):

    if file.endswith('.flac') and os.path.isfile(os.path.join(train_data_path,file)):
        file_path = os.path.join(train_data_path, file)
        
        with open(file_path, 'rb') as f:
            data = f.read()
        list_of_samples.append((data, file))
    else:
        print(f'Skipping {file}, is not a .flac file.')


dev_data_folder = 'Desktop/LA/LA/ASVspoof2019_LA_dev'
eval_data_folder = 'Desktop/LA/LA/ASVspoof2019_LA_eval'
save_folder = 'Griffith/Vishing_project/Notebook'
list_of_samples = []